##### 从Hub模型中心加载数据   
下例为MRPC中的glue数据集的加载过程

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

DatasetDict({
    train: Dataset({#训练集
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({#验证集
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({#测试集
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

可以看到每个数据集包含train set, validation set, test set。每个集合包含四个列：     
sentence1, sentence2, lable, idx    
详细含义如下：  

In [ ]:
{
    "sentence1": Value(dtype="string", id=None),
    "sentence2": Value(dtype="string", id=None),
    "label": ClassLabel(
        num_classes=2, names=["not_equivalent", "equivalent"], names_file=None, id=None
    ),
    "idx": Value(dtype="int32", id=None),
}

可以采用字典访问每个集合：

In [ ]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]


{
    "idx": 0,
    "label": 1,
    "sentence1": 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
    "sentence2": 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
}

##### tokenizer的token_type_ids      
当给tokenizer传入句子大于1个时，需要token_type_ids来区分得到的inputid是第几句，例子如下：

In [ ]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{
    "input_ids": [
        101,
        2023,
        2003,
        1996,
        2034,
        6251,
        1012,
        102,
        2023,
        2003,
        1996,
        2117,
        2028,
        1012,
        102,
    ],
    "token_type_ids": [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    "attention_mask": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
}


tokenizer.convert_ids_to_tokens(inputs["input_ids"])
[
    "[CLS]",
    "this",
    "is",
    "the",
    "first",
    "sentence",
    ".",
    "[SEP]",
    "this",
    "is",
    "the",
    "second",
    "one",
    ".",
    "[SEP]",
]
[
    "[CLS]",
    "this",
    "is",
    "the",
    "first",
    "sentence",
    ".",
    "[SEP]",
    "this",
    "is",
    "the",
    "second",
    "one",
    ".",
    "[SEP]",
]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

若直接按照tokenizer格式（如下），最终得到的是一个字典，键是input_ids, attention_mask, token_type_ids，其值是对应值的列表，这样要求在运行中需要有足够内存存储整个数据集。

In [ ]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)#返回字典，对内存要求高

但是dataset中的数据集是按照Apache Arrow格式存储，在运行时只需要运行接下来要用的数据，而不用加载全部数据集。   
可以使用Dataset.map()方法将数据保存为dataset格式，具体方式如下：

In [ ]:
def tokenize_function(example):
    #注意这里的tokenizer省略了padding参数，因为我们只需要填充到一个batch中最长句的大小即可
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})

Datesets库处理方式是对每个集合添加新的字段，每个字段对应预处理函数返回字典中的键。

##### 动态填充   
即将所有实例填充到该batch中最长元素的长度     
可以使用tansformers的DataCollatorWithPadding函数完成，其需要传入tokenizer参数，以确定应该用什么token进行填充

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

接下来进行该新函数的填充效果

In [ ]:
samples = tokenized_datasets["train"][:8]#取训练集的前8条
#删除列 idx ， sentence1 和 sentence2 ，因为不需要它们，而且删除包含字符串的列（我们不能用字符串创建张量）
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
#输出对应input_ids的长度
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

检查data_collator是否进行动态填充

In [ ]:

batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{
    "attention_mask": torch.Size([8, 67]),
    "input_ids": torch.Size([8, 67]),#包含8个样本，每个长度为67
    "token_type_ids": torch.Size([8, 67]),
    "labels": torch.Size([8]),
}

##### 使用TrainerAPI微调模型    
下例是以上所学对数据预处理的全部代码：

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Transformers提供了一个Trainer类，用来帮助微调过程，其难点在于准备Trainer.train()的环境，需要在GPU上运行，可以使用google Colab（需翻墙）     
以下是定义Trainer类的工作：


In [ ]:
from transformers import TrainingArguments
#TrainingArguments包含训练和评估中的参数，我们只需要给他传入用于保存训练后的模型以及训练中的checkpoin目录
training_args = TrainingArguments("test-trainer")
from transformers import AutoModelForSequenceClassification
#定义model，这里可能有警告，因为Bert在没有在句子分类中进行过预训练，故对应的head被丢弃，警告一些权重没有使用，有些权重被随机初始化
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,#默认为DataCollatorWithPadding
    tokenizer=tokenizer,
)

trainer.train()#进行微调

在微调中，每500步告诉一次损失。但不会告诉模型的质量如何，因为：     
我们没有告诉他进行评估。比如将 evaluation_strategy 设置为“ step ”（在每个 eval_steps 步骤评估一次）或“ epoch ”（在每个 epoch 结束时评估）。     
我们没有为 Trainer 提供一个 compute_metrics() 函数来计算上述评估过程的指标（否则评估将只会输出 loss，但这不是一个非常直观的数字）。     


##### 评估
如上所述，需要构建compute_metrics()函数，该函数必须接收一个 EvalPrediction 对象（它是一个带有 predictions 和 label_ids 字段的参数元组），并将返回一个字符串映射到浮点数的字典（字符串是返回的指标名称，而浮点数是其值）   
接下来是Trainer.predict()的过程：

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)
#predictions.predictions表示预测结果，predictions.label_ids表示在验证集中的真实标签
(408, 2) (408,)
#第一个表示predictions.predictions有408个样本，2表示两个类别的概率（是logits形式）
#第二个是真实标签，有408个

predict() 方法的输出另一个带有三个字段的命名元组: predictions label_ids 和 metrics metrics 字段将只包含所传递的数据集的损失,以及一些时间指标(总共花费的时间和平均预测时间)。   
若我们自己定义了compute_metrics函数，并将其传给Trainer，该字段还包括compute_metrics的结果

将上面代码的predictions.predictions中的logits形式通过softmax转化为概率

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

现在可以将preds与标签进行比对。    
compute_metrics可以使用Evaluate库中的指标

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

#结果可能与此不同，因为正如我们上面所言，bert没有对分类任务进行预训练，所以权重是随机的
{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}

将所有打包，就是compute_metrics函数，如下：

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


#将其加入Trainer类，完整过程如下：
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
#注意采用新的TrainingArguments,否则将会在旧的已训练过的模型上继续训练
#epoch表示在每个epoch结束时报告损失和指标
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

##### 完整训练（不实用Trainer方法，实现上述步骤）（基于Pytorch）      
    



#### 训练前准备   
删除不需要的列，如sentence1和sentence2   
将列名label改为labels（pytorch默认输入为labels）     
设置格式，使其返回Pytorch tensor

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names



["attention_mask", "input_ids", "labels", "token_type_ids"]

In [ ]:
#至此，可轻松定义数据加载器     
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    #shuffle表示是否对数据集进行随机打乱
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
#验证数据处理有无错误
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

#可能会有所不同，因为shuffle=true，且会自动填充到batch中最大长度
{'attention_mask': torch.Size([8, 65]),
 'input_ids': torch.Size([8, 65]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 65])}

In [ ]:
#进行模型实例化
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

#测试，传入batch
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)



tensor(0.5441, grad_fn=<NllLossBackward>) torch.Size([8, 2])

#### 开始训练循环
但仍缺少优化器和学习率调度器    
Trainer默认的优化器是AdamW

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

学习率调度器的定义，需要训练次数，即训练次数（epochs）*batch数量    
Trainer一般默认3个epochs

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    #这个参数指定了在训练开始时，学习率 线性增加 的步数，直到达到最终的学习率。通常用于优化器的学习率调度，特别是在使用学习率预热（warmup）策略时。
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

学习率预热，指让学习率在num_warmup_steps内线性增加到初始学习率，上例中将其设为0，即表示学习率直接从初始学习率开始。    
其作用可以用来避免梯度爆炸或更新步伐较大

In [ ]:
import torch
#将训练转向gpu
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device
device(type='cuda')

现在可以开始训练，但为了方便观察训练次数，使用tqdm库，它可以将循环可视化，显示类似于一个进度条

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()#转入训练模式
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}#将数据转入gpu
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()#计算梯度

        optimizer.step()#优化器进行更新
        lr_scheduler.step()#更新学习率
        optimizer.zero_grad()#清除梯度，pytorch会在反向传播中自动积累梯度
        progress_bar.update(1)#进度条更新

上述代码仍缺少一部分，即需要在每层循环中告知当前模型状态，故引入评估循环     
如上所述，利用Evaluate库引入compute_metric

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()#转入评估模式
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():#禁用梯度，因此时只是在评估，不需要计算梯度，可节省时间
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()


{'accuracy': 0.8431372549019608, 'f1': 0.8907849829351535}

In [ ]:
#上述过程总结（没有评估）
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

在上述基础上，引入Accelerate库，从而实现在多块gpu上分布训练

In [ ]:
+ from accelerate import Accelerator
  from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

+ accelerator = Accelerator()

  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
  optimizer = AdamW(model.parameters(), lr=3e-5)

#删减
- device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
- model.to(device)

+ train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
+     train_dataloader, eval_dataloader, model, optimizer
+ )

  num_epochs = 3
  num_training_steps = num_epochs * len(train_dataloader)
  lr_scheduler = get_scheduler(
      "linear",
      optimizer=optimizer,
      num_warmup_steps=0,
      num_training_steps=num_training_steps
  )

  progress_bar = tqdm(range(num_training_steps))

  model.train()
  for epoch in range(num_epochs):
      for batch in train_dataloader:
-         batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(**batch)
          loss = outputs.loss
-         loss.backward()
+         accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)

In [ ]:
#利用Accelerate完整代码
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

#####  关于其accelerate的设置

In [ ]:
！accelerate config
！accelerate launch train.py

In [ ]:
#若是在notebook中
from accelerate import notebook_launcher

notebook_launcher(training_function)